<a href="https://colab.research.google.com/github/Keerthibalraj/Day_2_CBT_Course/blob/main/DAY_2_CBT_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORT LIBRARY**

In [ ]:
import os
import sys
from sys import stdout
import logging

import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import clear_output

import scipy.io as sio
from scipy.signal import savgol_filter
import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score , KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv1D, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback,  ModelCheckpoint
from tensorflow.keras.utils import plot_model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load data
data = **missing code**
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'DM_cal', 'DM_test', 'Sp_cal', 'Sp_test', 'wave', 'SP_all_test', 'SP_all_train'])


In [ ]:
# Visualize the dataset
plt.figure(figsize=(10,6))

plt.subplot(221)
plt.title('Train data')
plt.plot(data['wave'].T,data['Sp_cal'].T)
plt.ylabel('Amplitude (a.u)')
plt.xlabel('Wavelength (nm)')

plt.subplot(222)
plt.title('Test data')
plt.plot(data['wave'].T,data['Sp_test'].T)
plt.ylabel('Amplitude (a.u)')
plt.xlabel('Wavelength (nm)')

plt.subplot(223)
plt.plot(data['DM_cal'])
plt.ylabel('Dry Matter (%)')
plt.xlabel('Sample number')

plt.subplot(224)
plt.plot(data['DM_test'])
plt.ylabel('Dry Matter (%)')
plt.xlabel('Sample number')
plt.tight_layout()
plt.show()

In [ ]:
## Spectra (x) and target variable (Y)

## Convert data type to float32 for better inter-operability with TensorFlow
x_train=data['SP_all_train'].astype(np.float32)
y_train=data['DM_cal'].astype(np.float32)
x_test=data['SP_all_test'].astype(np.float32)
y_test=data['DM_test'].astype(np.float32)


In [ ]:
## Spliting the full train set into calib and tuning subsets. It is important to set the 'random_state'
## parameter to a fixed value in order to guarentee that each time you run the experiment, the data is
## split the same way
X_train, X_val, Y_train, Y_val=** missing code**
## The wavelenghts for the XX axis when we plot the spectra
x_scale=data['wave'].astype(np.float32).reshape(-1,1)

## Check for dimensions
print('Data set dimensions ----------------------------')
print('Full Train set dims X Y = {}\t{}'.format(x_train.shape, y_train.shape))
print('Training set dims X Y = {}\t{}'.format(X_train.shape, Y_train.shape))
print('Validation set dims X Y = {}\t{}'.format(X_val.shape, Y_val.shape))
print('Test set dims X Y = {}\t{}'.format(x_test.shape, y_test.shape))
print('wavelengths number = {}'.format(np.shape(x_scale)))

In [ ]:
## Visual data sanity checking for the test set
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
plt.plot(x_test[:50,:].T,'k')
plt.title('First 50 spectra from the tuning sample')
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel('Spectra intensity')

plt.subplot(1,2,2)
plt.title('Y labels')
plt.plot(y_test,'k.')
plt.xlabel('Sample number')
plt.ylabel('Dry Matter (%)')
plt.show()

In [ ]:
## Since the test set is unknown (we are not suppose to have access to it during the
## optimization of the model) the scalling process should take this into account. We
## have to define a scaler based only on the train data, and apply it to the test data.

def standardize_column(X_train, X_val, x_test):
    ## We train the scaler on the full train set and apply it to the other datasets
    scaler = **missing data**
    ## for columns we fit the scaler to the train set and apply it to the test set
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(x_test)
    return [X_train_scaled, X_val_scaled, X_test_scaled]

In [ ]:
## Standardize on columns
X_train_scaled, X_val_scaled, X_test_scaled= **missing data**

In [ ]:
plt.figure(figsize=(18,4))
## left panel
plt.subplot(121)
plt.title('A', fontsize=18)
plt.plot(x_test[:50].T)
plt.ylabel('Unscaled amplitude')
plt.xlabel('Feature number')
## vertical dashed lines separating the types of preprocessings that were concatenated
plt.vlines(np.arange(103,600,103), ymin=-1.1, ymax=5.1, color='k',ls='--',lw=1)
plt.xlim(-5,625)
plt.ylim(-1,5)

## right panel
plt.subplot(122)
plt.title('B', fontsize=18)
plt.plot(X_test_scaled[:50].T)
plt.ylabel('Standardized amplitude')
plt.xlabel('Feature number')
plt.vlines(np.arange(103,600,103), ymin=-7.6, ymax=5.6, color='k',ls='--',lw=1)
plt.xlim(-5,625)
plt.ylim(-7.5,5.5)

plt.show()

In [ ]:
**missing package**

In [ ]:
from lazypredict.Supervised import LazyRegressor

reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train_scaled, X_test_scaled, Y_train, y_test)

print(models)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Reshape, GaussianNoise
from keras.callbacks import ReduceLROnPlateau

In [ ]:
def init_model(filter_size, reg_beta):
**missing data**


    return model_cnn

In [ ]:
## Function to compute metrics and make prediction plots
def plot_prediction(X_calib, Y_calib, X_valid, Y_valid, X_test, Y_test, Y_calib_pred, Y_valid_pred, Y_test_pred, savefig=False, figname=None):

    ## Compute train error scores
    score_p0 = r2_score(Y_calib, Y_calib_pred)
    mse_p0 = mean_squared_error(Y_calib, Y_calib_pred)
    rmse_p0 = np.sqrt(mse_p0)

    ## Compute valid error scores
    score_p1 = r2_score(Y_valid, Y_valid_pred)
    mse_p1 = mean_squared_error(Y_valid, Y_valid_pred)
    rmse_p1 = np.sqrt(mse_p1)

    ## Compute test error scores
    score_p2 = r2_score(Y_test, Y_test_pred)
    mse_p2 = mean_squared_error(Y_test, Y_test_pred)
    rmse_p2 = np.sqrt(mse_p2)


    print('ERROR METRICS: \t TRAIN \t\t VALID \t\t TEST')
    print('------------------------------------------------------')
    print('R2: \t\t %5.3f \t\t %5.3f \t\t %5.3f'  % (score_p0, score_p1, score_p2 ))
    print('RMSE: \t\t %5.3f \t\t %5.3f \t\t %5.3f' % (rmse_p0, rmse_p1,  rmse_p2))


    ## Plot regression for PLS predicted data
    rangey = max(Y_test) - min(Y_test)
    rangex = max(Y_test_pred) - min(Y_test_pred)

    fig=plt.figure(figsize=(6,6))
    z = np.polyfit(np.ravel(Y_test), np.ravel(Y_test_pred), 1)
    ax = plt.subplot(aspect=1)
    ax.scatter(Y_test,Y_test_pred,c='k',marker='o',s=20, alpha=0.6)
    ax.plot(Y_test, z[1]+z[0]*Y_test, c='blue', linewidth=2,label='linear fit')
    ax.plot(Y_test, Y_test, 'k--', linewidth=1.5, label='y=x')
    plt.ylabel('Predicted DM')
    plt.xlabel('Measured DM')
    plt.title('Prediction from CNN')
    plt.legend(loc=4)


    # Print the scores on the plot
    plt.text(min(Y_test_pred)+0.02*rangex, max(Y_test)-0.1*rangey, 'R$^{2}=$ %5.3f'  % score_p2)
    plt.text(min(Y_test_pred)+0.02*rangex, max(Y_test)-0.15*rangey, 'RMSE: %5.3f' % rmse_p2)
    if savefig==True:
        plt.savefig(figname, dpi=96)
        print('Figure saved')
    else:
        plt.show()
    return

In [ ]:
## Initialize a new CNN with a filter size of 5 points and L2 reg. = 0.0006 (weak regularization)
model_cnn = init_model(5, 0.0006)

## Print the summary of the model
print(model_cnn.summary())

In [ ]:
## Plot the architecture of the CNN
## The plot_model() from Keras requires that graphviz and pydot are installed in your system.
## If you dont't have these libraries installed, skip this cell for now and browse for the image "base_cnn_regression.png" in the repository to see the result
plot_model(**missingdata**)

In [ ]:
########### DEFINE HYPERPARAMETERS AND INSTANTIATE THE MODEL #####################
## Define the batch size (number of sample to train for each pass into the CNN)
BATCH = 256
## Define the filter size
FILTER_SIZE = 5
## Define the L2 reg.
L2_BETA = 0.0006

## initialize the model
model_cnn=init_model(FILTER_SIZE, L2_BETA)

In [ ]:
## Run this base model once to see if its training correctly
## This can be viewed as a baseline that hyperparameter optimization should improve

tf.keras.backend.clear_session()

In [ ]:
########### COMPILE MODEL WITH ADAM OPTIMIZER #####################################

## Heuristic that sets the learning rate
LR=** missing data**
print('Adam learning rate = {}'.format(LR))

## Define the model name for saving purposes
MODEL_NAME = 'base_regression_model.h5'

## Compile the model defining the optimizer, the loss function and the metrics to track during training
model_cnn.compile(**missing data**)

Adam learning rate = 0.01


In [ ]:
########### DEFINE USEFUL CALLBACK FUNCTIONS #####################################

## The following are several callbacks that can be used during training to improve its efficiency and decrease overfit
## 1) Stop the training if it does not improve
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=50, mode='auto', restore_best_weights=True)

## 2) Reduce learning rate dynamically
rdlr = ReduceLROnPlateau(patience=25, factor=0.5, min_lr=1e-6, monitor='val_loss', verbose=0)

## 3) Save the best weights into file
checkpointer = ModelCheckpoint(filepath=MODEL_NAME, verbose=1, save_best_only=True)


In [ ]:
########### TRAIN THE MODEL #####################################################
## Train the model for 450 epochs on the x_cal data while monitoring the models performance on the x_tuning data.
## The data in the tuning set is not used for actual training. It is only used to measure model performance.
h1 = model_cnn.fit(**missing data**)

## Clear the session. This is important to ensure that on the next training session, the weights are properly initialized.
## On the other hand if one need to continue a training session for longer, this can just be commented.
tf.keras.backend.clear_session()

In [ ]:
## If you used Livelossplot, you can skip this. Otherwise we can take a look at the training process by plotting the
## models history.
plt.figure(figsize=(8,4))
plt.plot(h1.history['loss'], label='Calibration set loss')
plt.plot(h1.history['val_loss'], label='Tunning set loss')
plt.yscale('log')
plt.ylabel('Loss')
plt.xlabel('Epochs')
# plt.ylim(0.5,1)
plt.legend()
## In case you used ReduceLROnPlateau() you can plot the lr as well
ax2 = plt.gca().twinx()
ax2.plot(h1.history['lr'], color='r', ls='--')
ax2.set_ylabel('learning rate',color='r')
plt.tight_layout()
plt.show()
# plt.savefig('train_tun_loss.png', dpi=96)

In [ ]:
## Initialize a model with the best found hyperparameters
model_cnn = **missing data**

## Load the weights of the pre-trained / saved corresponding model
## load pre-computed model weights into model_cnn
model_cnn.load_weights("/content/base_regression_model.h5")

## Compute metrics for Training, validation and Test sets
y_train_pred = model_cnn.predict(X_train_scaled)
y_val_pred3 = model_cnn.predict(X_val_scaled)
y_test_pred3  = model_cnn.predict(X_test_scaled)



In [ ]:
## Plot the prediction metrics using the custom function "plot_prediction", defined in the initial Help section.
plot_prediction(X_train_scaled,Y_train, X_val_scaled, Y_val, X_test_scaled, y_test, y_train_pred, y_val_pred3, y_test_pred3, savefig=False, figname='prediction_base_model1.png')
